In [39]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [40]:
# read in the data
pedestrian_network = gpd.read_file('../data/cleaned-pednetwork/cleaned-pednetwork-full/cleaned-pednetwork-full.shp')

# 1. Waytype

In [41]:
#check value counts for fclass do not drop null
pedestrian_network['fclass'].value_counts(dropna=False)

fclass
footway        21715
crossing        9959
service          989
residential      698
steps            493
tertiary         253
pedestrian       239
secondary        229
cycleway         184
m_service         96
primary           32
trunk             22
path              13
m_footway         10
track              1
Name: count, dtype: int64

# 2. Sidwalk Width

In [42]:
# read in the sidewalk width data
sidewalk_width = pd.read_csv('../data/pednetwork-attributes/sidewalks/paving-and-width.csv')
sidewalk_width

,pednet_pid,surface_t,width
0,0,concrete,72.0
1,1,concrete,106.0
2,2,concrete,61.0
3,3,concrete,72.0
4,4,concrete,72.0
...,...,...,...
34924,34924,concrete,72.0
34925,34925,concrete,72.0
34926,34926,concrete,72.0
34927,34927,concrete,72.0


In [43]:
#join on pednet_pid
pedestrian_network = pedestrian_network.merge(sidewalk_width, on='pednet_pid', how='left')

# 3. Sidewalk surface type

In [44]:
pedestrian_network['surface_t'].unique()

array(['concrete', 'gravel', 'asphalt', 'brick'], dtype=object)

# 4. Number of steps

In [45]:
# read in the number of steps data
number_of_steps = pd.read_csv('../data/pednetwork-attributes/sidewalks/number-of-steps.csv')

In [46]:
#join on pednet_id
pedestrian_network = pedestrian_network.merge(number_of_steps, on='pednet_id', how='left')

# 5. Slope

In [47]:
# add a column for z difference
pedestrian_network['z_d'] = abs(pedestrian_network['z-start'] - pedestrian_network['z-end'])
#change the z_d to meters to feet
pedestrian_network['z_d'] = pedestrian_network['z_d'] * 3.28084

In [48]:
# create a new column for the length of the pedestrian network
pedestrian_network['length'] = pedestrian_network['geometry'].length

In [49]:
#calculate the slope in percent
pedestrian_network['slope'] = pedestrian_network['z_d'] / pedestrian_network['length']

# 6. Crossings

In [50]:
#read in crossing traffic flow data
crossing_traffic_flow = pd.read_csv('../data/pednetwork-attributes/crosswalks/crossings-traffic-flow.csv')

In [51]:
# join on pednet_id
pedestrian_network = pedestrian_network.merge(crossing_traffic_flow, on='pednet_id', how='left')

In [52]:
# change the column orders
pedestrian_network = pedestrian_network[['pednet_id', 'pednet_pid', 'type', 'fclass', 'surface_t', 'width', 'number-of-steps', 'z-start', 'z-end', 'z_d', 'length', 'slope', 'crossing_t', 'traffic_flow', 'geometry']]

In [53]:
#change column names from 'number-of-steps' to 'number_of_steps'
pedestrian_network = pedestrian_network.rename(columns={'number-of-steps': 'steps'})
#change column names from z_d to z_difference
pedestrian_network = pedestrian_network.rename(columns={'z_d': 'z_delta'})

In [54]:
pedestrian_network

,pednet_id,pednet_pid,type,fclass,surface_t,width,steps,z-start,z-end,z_delta,length,slope,crossing_t,traffic_flow,geometry
0,0.0,0,ped,crossing,concrete,72.0,NaN,15.400000,16.200001,2.624676,71.188703,0.036869,m,low,"LINESTRING (1277935.51 238384.953, 1277942.83 ..."
1,1.0,1,ped,footway,concrete,106.0,NaN,16.200001,16.200001,0.000000,16.675574,0.000000,0,NaN,"LINESTRING (1277978.83 238310.174, 1277966.394..."
2,2.0,2,ped,footway,concrete,61.0,NaN,10.500000,16.200001,18.700791,458.294269,0.040805,0,NaN,"LINESTRING (1278374.334 238398.46, 1278317.326..."
3,3.0,3,ped,steps,concrete,72.0,1.0,11.600000,11.600000,0.000000,17.555658,0.000000,0,NaN,"LINESTRING (1284223.335 235156.549, 1284211.29..."
4,4.0,4,ped,steps,concrete,72.0,5.0,124.599998,118.900002,18.700778,79.474146,0.235306,0,NaN,"LINESTRING (1266761.001 234328.359, 1266840.43..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34928,32959.0,34924,ped,footway,concrete,72.0,NaN,68.199997,68.199997,0.000000,10.709117,0.000000,0,NaN,"LINESTRING (1262631.523 239983.092, 1262641.40..."
34929,32960.0,34925,road,service,concrete,72.0,NaN,65.099998,62.700001,7.874008,60.364828,0.130440,0,NaN,"LINESTRING (1262862.932 240024.343, 1262868.78..."
34930,32961.0,34926,road,service,concrete,72.0,NaN,62.700001,65.099998,7.874008,155.739448,0.050559,0,NaN,"LINESTRING (1262918.344 240036.228, 1262921.4 ..."
34931,32962.0,34927,road,service,concrete,72.0,NaN,61.599998,62.700001,3.608932,344.187381,0.010485,0,NaN,"LINESTRING (1263190.244 239825.195, 1263057.2 ..."


In [55]:
# save network to file
pedestrian_network.to_file('../data/cleaned-pednetwork/cleaned-pednetwork-full-attributes/cleaned-pednetwork-full-attributes.shp')

/var/folders/zm/x5dzdlrs5mlbxrm7z5g6p05r0000gn/T/ipykernel_51648/2680768217.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  pedestrian_network.to_file('../data/cleaned-pednetwork/cleaned-pednetwork-full-attributes/cleaned-pednetwork-full-attributes.shp')
/Users/chuli/opt/anaconda3/envs/project-sidewalk/lib/python3.10/site-packages/pyogrio/raw.py:530: RuntimeWarning: Normalized/laundered field name: 'traffic_flow' to 'traffic_fl'
  ogr_write(
